# 5. Open types

Consider again the (modified) example from Section 2, in which eligibility for acceptance into St. John's secondary school was to specified:

In [1]:
Fact applicant
Fact primary-school Identified by StMary, StGeorge
Fact gpa Identified by 1..4
Fact diploma Identified by primary-school * applicant * gpa 
Fact application Identified by applicant * diploma
Fact accepted Identified by applicant

Act accept-application Related to application
  Holds when application             
  Conditioned by application.diploma && diploma.gpa >= 3
  Creates accepted()
.
+applicant(Alice).
+applicant(Bob).
+applicant(Chloe).
+diploma(StMary, Alice, 3).
+diploma(StGeorge, Bob, 3).
+diploma(StGeorge, Chloe, 2).
?Enabled(accept-application(StJohn, application(Alice, diploma(StMary, Alice, 3)))).
?Enabled(accept-application(StJohn, application(Bob, diploma(StGeorge, Bob, 3)))).
?!Enabled(accept-application(StJohn, application(Chloe, diploma(StGeorge, Chloe, 2)))).

+applicant("Alice")
+applicant("Bob")
+applicant("Chloe")
+diploma(primary-school("StGeorge"),applicant("Bob"),gpa(3))
+diploma(primary-school("StGeorge"),applicant("Chloe"),gpa(2))
+diploma(primary-school("StMary"),applicant("Alice"),gpa(3))


Query failed
Query failed
Query success


The previous code fragment checks whether Alice, Bob and/or Chloe can be accepted into the school based on their diplomas and whether they are already considered applicants of the school. These two kinds of facts of the case -- recognised diplomas and being an applicant -- are very different in nature in that the school can be expected to know the latter kind (based on an application letter, for example) but may have to rely on some external authority to determine whether the diplomas are valid and indeed handed out by the corresponding primary-school.

If we imagine an eFLINT service running within the local network of St. John's, then the information about applicants can come from within that network, for example, by triggering the action `send-application-letter`.

In [2]:
Act send-application-letter Actor applicant Related to diploma
  Holds when !applicant // always enabled for non-applicants
  Creates application()
.
send-application-letter(David, diploma(StMary, David, 4))

+application(applicant("David"),diploma(primary-school("StMary"),applicant("David"),gpa(4)))


Executed transition: send-application-letter(applicant("David"),diploma(primary-school("StMary"),applicant("David"),gpa(4)))


When specifying the policy of St. John's in eFLINT, we can decide we have confidence in the school's application-processing software to reliably send updates about received letters to the eFLINT service (perhaps because both services are under our full control). In these conditions, it is safe to apply the **closed-world assumption** to instances of the type `applicant` -- if the eFLINT services does not know `X` applied, then `X` did indeed not apply. 

The same cannot be said for the validity of diplomas and we may wish to verify that the primary-school did indeed issue this exact diploma. To support such situations, types can be declared as being 'open' (with the `Open` modifier, see subsection 1.7).

In [3]:
Open Fact diploma Identified by primary-school * applicant * gpa

To determine whether a particular instance of an open type holds true, it is checked whether it is stated being as true (or false) within the *input* provided alongside the phrase being executed, then whether it is stated as being true (or false) within the current knowledge base, then whether it can be derived as holding true, and finally, if neither of the aforementioned yielded a truth value, then a "missing input" exception is thrown.

In [4]:
?Enabled(accept-application(StJohn, application(Alice, diploma(StMary, Alice, 3)))).
?Enabled(accept-application(StJohn, application(Bob, diploma(StGeorge, Bob, 3)))).
?!Enabled(accept-application(StJohn, application(Chloe, diploma(StGeorge, Chloe, 2)))).
?!Enabled(accept-application(StJohn, application(David, diploma(StMary, David, 4)))).

Query failed
Query failed
Query success
missing input assignment for: diploma(primary-school("StMary"),applicant("David"),gpa(4))


This information feeds back into the execution context (e.g. the eFLINT service) and might trigger the execution of a handler to provide the missing information (e.g. a notification sent to an employee of the school to reach out to primary school for validation).

In the `eflint-repl`, a recursive prompt starts to ask the programmer for any assignments for missing input whenever instances of open types were evaluated during the execution of a phrase. This prompt repeatedly asks the programmer to provide the input for instances of open types that are required to complete the execution of the phrase until all missing input is given.

A future version of the Jupyter Kernel for eFLINT may support one or more approaches to providing input and handling missing input.

Note that the input mechanism can be used to replace the internal knowledge base of eFLINT entirely, effectively creating an external knowledge base (e.g. in the form of a persistent database).